# Unit 2 - Part 2a: The Anatomy of a Prompt

## 1. Introduction: Stochasticity (Randomness)

Why does the AI give different answers? Because it is **Stochastic** (Random).

It predicts the NEXT TOKEN based on probability.

### Visualizing the Prediction
Input: `"The sky is..."`

| Word | Probability | Selected? (Temp=0) | Selected? (Temp=1) |
|------|-------------|--------------------|--------------------|
| Blue | 80% | ✅ | ❌ |
| Gray | 15% | ❌ | ✅ |
| Green| 1% | ❌ | ❌ |

Prompt Engineering is the art of **manipulating these probabilities**.

In [1]:
%pip install python-dotenv --upgrade --quiet langchain langchain-google-genai

In [2]:
# Setup
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")

# Using Low Temp for consistent comparison
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.0)

Enter your Google API Key: ··········


## 2. The CO-STAR Framework (simplified)

A good prompt usually has:
1.  **C**ontext (Who are you? Who acts?)
2.  **O**bjective (What is the task?)
3.  **S**tyle (Formal? Funny?)
4.  **T**one (Empathetic? Direct?)
5.  **A**udience (Who is reading this?)
6.  **R**esponse Format (JSON? List?)

Let's compare a **Lazy Prompt** vs a **CO-STAR Prompt**.

In [3]:
# The Task: Reject a candidate for a job.
task = "Write a essay about birds."

print("--- LAZY PROMPT ---")
print(llm.invoke(task).content)

--- LAZY PROMPT ---
## The Feathered Symphony: A Testament to Life's Enduring Wonder

From the smallest hummingbird to the majestic eagle, birds are a ubiquitous and captivating presence in nearly every corner of our planet. More than mere inhabitants, they are vibrant threads in the tapestry of life, embodying grace, resilience, and an astonishing array of adaptations that have allowed them to thrive across diverse ecosystems. Their very existence is a symphony of wonder, a constant reminder of nature's artistry and the delicate balance that sustains our world.

Perhaps the most iconic characteristic of birds is their mastery of the skies. The miracle of flight, a feat of aerodynamic engineering perfected over millions of years, allows them to traverse vast distances, escape predators, and access food sources unavailable to ground-bound creatures. Their lightweight, hollow bones, powerful musculature, and intricately designed feathers are a testament to evolutionary brilliance. Watchi

## 3. Hallucination vs. Creativity

Did the model make up a reason?
Since we didn't give it facts, it **Predicted the most likely reason** (Usually "Experience" or "Volume of applications").

**This is NOT a bug.** It is a feature. The model is *completing the pattern* of a rejection email.

In [4]:
structured_prompt = """
# Context
You are an HR Manager at a quirky startup called 'Human Beings'.

# Objective
Write a acceptance email to a candidate named Bob.

# Constraints
1. Be extremely brief (under 50 words).
2. Do NOT say 'the role changed'.
3. Sign off with 'Keep flying'.

# Output Format
Plain text, no subject line.
"""

print("--- STRUCTURED PROMPT ---")
print(llm.invoke(structured_prompt).content)

--- STRUCTURED PROMPT ---
Hi Bob,

Fantastic news! We're thrilled to offer you a position at Human Beings. Your unique talents are a perfect fit for our team. We're excited to have you join us. More details will follow soon.

Keep flying,
[Your Name]
HR Manager, Human Beings


## 4. Key Takeaway: Ambiguity is the Enemy

Every piece of information you leave out is a gap the model MUST fill with probability.
- If you don't say "Be brief", it picks the most probable length (Avg email length).
- If you don't say "Be rude", it picks the most probable tone (Polite/Neutral).

## Assignment

Write a structured prompt to generate a **Python Function**.
- **Context:** You are a Senior Python Dev.
- **Objective:** Write a function to reverse a string.
- **Constraint:** It must use recursion (no slicing `[::-1]`).
- **Style:** Include detailed docstrings.

# Unit 2 - Part 2b: Zero-Shot to Few-Shot

## 1. Introduction: In-Context Learning

How does the model learn without training?
This is called **In-Context Learning**.

### The Attention Mechanism (Flowchart)
When you ask a question, the model "looks back" at the previous text to find patterns.

```mermaid
graph TD
    Input[Current Input: 'Angry + Hungry'] -->|Attention Query| History
    subgraph History [The Prompt Examples]
        Ex1[Ex1: Breakfast + Lunch = Brunch]
        Ex2[Ex2: Chill + Relax = Chillax]
    end
    History -->|Pattern Found: Mix words & define| Prediction[Output: Hangry]
```

In [5]:
# Setup
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.5)

## 2. Zero-Shot (No Context)

The model relies purely on its training data.

In [6]:
prompt_zero = "Combine 'Angry' and 'Sergio Ramos'into a funny new word."
print(f"Zero-Shot: {llm.invoke(prompt_zero).content}")

Zero-Shot: Here are a few funny options, playing on different aspects:

1.  **Rage-mos:** (Simple, direct, and captures his fiery nature)
2.  **Furi-amos:** (Blends "furious" with Ramos, sounds a bit like a Latin spell)
3.  **Ram-page-ous:** (Plays on "rampage" and his name, implying chaotic anger)
4.  **Serg-rage:** (Combines "Sergio" with "rage" for a punchy sound)
5.  **Anger-gio:** (A bit more playful, sounds like a specific kind of anger)

My personal favorite for a quick laugh is **Rage-mos** or **Ram-page-ous**!


## 3. Few-Shot (Pattern Matching)

We provide examples. The Attention Mechanism attends to the **Structure** (`Input -> Output`) and the **Tone** (Sarcasm).

In [7]:
prompt_few = """
Combine two words into a creative blended word.

Examples:

Words: cold + coffee
Output: Coffreeze

Words: smoke + fog
Output: Smog

Words: breakfast + lunch
Output: Brunch

Now combine:

Words: angry + "sergio"
Output:
"""

print("Few-shot:", llm.invoke(prompt_few).content)

Few-shot: **Sergry**


## 4. Critical Analysis

If you provide **bad examples**, the model will learn the **bad pattern**.
This is why Data Quality in your prompt is just as important as code quality.

# Unit 2 - Part 2c: Advanced Templates & Theory

## 1. Theory: Engineering vs. Training

### Hard Prompts (Prompt Engineering)
- **What:** You change the text input.
- **Cost:** Cheap, fast, easy to iterate.
- **Use Case:** Prototyping, General tasks.

### Soft Prompts (Fine Tuning)
- **What:** You change the model's internal weights (mathematically).
- **Cost:** Expensive, slow, needs data.
- **Use Case:** Domain specificity (Medical, Legal), Behavioral change.

In [8]:
# Setup
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

## 2. Dynamic Few-Shotting

If you have 1000 examples, you can't fit them all in the context window.
We use a **Selector** to pick the best ones.

### The Selector Flow (Flowchart)
```mermaid
graph LR
    Input[User Input] -->|Semantic Search| Database[Example Database]
    Database -->|Top 3 Matches| Selector
    Selector -->|Inject| Prompt
```

In [9]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# Format of each example
example_fmt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}")
])

# Example database
examples = [
    {
        "input": "I want to stay in bed all day but I also feel guilty for being unproductive.",
        "output": "Restlessponsible – A blended term describing the conflict between craving rest and feeling responsible for unfinished tasks."
    },
    {
        "input": "She keeps checking her phone even though she knows no new messages have arrived.",
        "output": "Notifxious – A playful blend capturing the anxious anticipation of notifications that may not exist."
    },
    {
        "input": "He laughed at the joke but deep down he was slightly offended.",
        "output": "Laughended – A creative expression describing the awkward mix of amusement and mild offense."
    },
]


few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_fmt,
    examples=examples
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a Corpo-Speak Translator"),
    few_shot_prompt,
    ("human", "{text}")
])

chain = final_prompt | llm

print(chain.invoke({"text": "This app is annoying ."}).content)


**Appstration** – A blend of "app" and "frustration," describing the feeling of annoyance or irritation caused by a poorly designed or malfunctioning application.


## 3. Analysis

Using `FewShotChatMessagePromptTemplate` creates a clean separation between instructions and data. This helps the Attention Mechanism focus on the right things.

In [10]:
from langchain_core.prompts import ChatPromptTemplate

structured_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a Principal Python Engineer who writes highly maintainable, "
     "well-tested, production-grade Python code with strong type hints."),

    ("human",
     "Implement a Python function that reverses a string.\n\n"
     "Technical Requirements:\n"
     "- You MUST use recursion\n"
     "- You MUST NOT use slicing\n"
     "- Do not use built-in reverse utilities\n"
     "- Include type hints\n"
     "- Handle edge cases properly\n\n"
     "Code Quality Requirements:\n"
     "- Add comprehensive docstrings (Google style)\n"
     "- Keep it clean and readable\n"
     "- Follow PEP8 standards\n"
     "- Include example usage")
])

chain = structured_prompt | llm

print(chain.invoke({}).content)


As a Principal Python Engineer, I prioritize clarity, correctness, and adherence to specified constraints. This implementation uses a nested helper function to manage the recursive state (the current index) without exposing it to the public API, ensuring a clean and intuitive interface. The "no slicing" constraint is met by using index-based character access and integer arithmetic for the recursive calls.

```python
from typing import TypeGuard

def reverse_string(s: str) -> str:
    """Reverses a string using recursion without using slicing or built-in utilities.

    This function reverses the input string by recursively processing characters
    from left to right and appending them to the result of the reversed
    subsequent characters. It achieves this by using a helper function that
    manages the current index, thereby avoiding string slicing.

    Args:
        s: The input string to be reversed.

    Returns:
        The reversed string.

    Raises:
        TypeError: If th